In [2]:
# test-page

import pandas as pd
import requests
import io

print("Python siap 🚀")

Python siap 🚀


In [14]:
# listing the districts and cities code based on DJPK

!pip install beautifulsoup4 lxml

In [17]:
# listing the codes

import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://djpk.kemenkeu.go.id/portal/data/tkdd"
tahun = 2023

pemda_rows = []

for _, row in df_provinsi.iterrows():
    kode_prov = str(row["kode_provinsi"])
    nama_prov = row["nama_provinsi"]

    params = {
        "tahun": tahun,
        "provinsi": kode_prov,
        "pemda": "--"      # let this empty, the dropdown will appear
    }

    print(f"Ambil daftar pemda untuk provinsi {kode_prov} - {nama_prov} ...")
    resp = requests.get(base_url, params=params, timeout=60)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "lxml")

    pemda_select = soup.find("select", {"name": "pemda"}) or soup.find("select", {"id": "pemda"})
    if pemda_select is None:
        print(f"   ⚠️ Tidak ditemukan dropdown pemda untuk provinsi {kode_prov}, skip.")
        continue

    for opt in pemda_select.find_all("option"):
        val = (opt.get("value") or "").strip()
        text = opt.text.strip()
        if not val or val == "--":
            continue

        pemda_rows.append({
            "kode_provinsi": kode_prov,
            "nama_provinsi": nama_prov,
            "kode_pemda": val,
            "nama_pemda": text
        })

df_pemda = pd.DataFrame(pemda_rows)
df_pemda


Ambil daftar pemda untuk provinsi 01 - Provinsi Aceh ...
Ambil daftar pemda untuk provinsi 02 - Provinsi Sumatera Utara ...
Ambil daftar pemda untuk provinsi 03 - Provinsi Sumatera Barat ...
Ambil daftar pemda untuk provinsi 04 - Provinsi Riau ...
Ambil daftar pemda untuk provinsi 05 - Provinsi Jambi ...
Ambil daftar pemda untuk provinsi 06 - Provinsi Sumatera Selatan ...
Ambil daftar pemda untuk provinsi 07 - Provinsi Bengkulu ...
Ambil daftar pemda untuk provinsi 08 - Provinsi Lampung ...
Ambil daftar pemda untuk provinsi 09 - Provinsi DKI Jakarta ...
Ambil daftar pemda untuk provinsi 10 - Provinsi Jawa Barat ...
Ambil daftar pemda untuk provinsi 11 - Provinsi Jawa Tengah ...
Ambil daftar pemda untuk provinsi 12 - Provinsi DI Yogyakarta ...
Ambil daftar pemda untuk provinsi 13 - Provinsi Jawa Timur ...
Ambil daftar pemda untuk provinsi 14 - Provinsi Kalimantan Barat ...
Ambil daftar pemda untuk provinsi 15 - Provinsi Kalimantan Tengah ...
Ambil daftar pemda untuk provinsi 16 - Provin

,kode_provinsi,nama_provinsi,kode_pemda,nama_pemda
0,01,Provinsi Aceh,00,Provinsi Aceh
1,01,Provinsi Aceh,01,Kab. Aceh Barat
2,01,Provinsi Aceh,02,Kab. Aceh Besar
3,01,Provinsi Aceh,03,Kab. Aceh Selatan
4,01,Provinsi Aceh,04,Kab. Aceh Singkil
...,...,...,...,...
541,38,Provinsi Papua Barat Daya,02,Kota Sorong
542,38,Provinsi Papua Barat Daya,03,Kab. Raja Ampat
543,38,Provinsi Papua Barat Daya,04,Kab. Sorong Selatan
544,38,Provinsi Papua Barat Daya,05,Kab. Maybrat


In [18]:
df_pemda.to_excel("djpk_kode_pemda.xlsx", index=False)
print("Selesai simpan kode pemda ke djpk_kode_pemda.xlsx")


Selesai simpan kode pemda ke djpk_kode_pemda.xlsx


In [25]:
# Automatically download the data from the website

import requests
from bs4 import BeautifulSoup
import pandas as pd
import io
import os
import shutil
import re
import urllib3

# Turn off the warning "InsecureRequestWarning", cuz I use verify=False
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# ============================================================
# >>> Need to be replaced only <<<
tahun = 2025   # replace with the year I want to download
# ============================================================

base_page_url = "https://djpk.kemenkeu.go.id/portal/data/tkdd"
base_csv_url = "https://djpk.kemenkeu.go.id/portal/csv"

# One session for stable connection
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0 Safari/537.36"
})

def safe_filename(name: str) -> str:
    """Bikin nama file aman (spasi tetap, Kab./Kota tetap)."""
    name = re.sub(r'[\\/*?:"<>|]', '', name)  # delete the forbidden signs based on, for ready files
    return name.strip()

print(f"Mengambil daftar provinsi untuk tahun {tahun} ...")

# --- 1. Take the dropdown of 'provinsi' on the main page ---
params_root = {"tahun": tahun, "provinsi": "--", "pemda": "--"}
resp_root = session.get(base_page_url, params=params_root, timeout=60, verify=False)
resp_root.raise_for_status()

soup_root = BeautifulSoup(resp_root.text, "lxml")
prov_select = soup_root.find("select", {"name": "provinsi"}) or soup_root.find("select", {"id": "provinsi"})

prov_list = []
for opt in prov_select.find_all("option"):
    val = (opt.get("value") or "").strip()
    text = opt.text.strip()
    if not val or val == "--":
        continue
    prov_list.append({"kode_prov": val, "nama_prov": text})

print(f"➜ Ditemukan {len(prov_list)} provinsi.\n")

# Folder root for the kab/kota files in this year
root_folder = f"tkdd_{tahun}_kab_kota"
os.makedirs(root_folder, exist_ok=True)

gagal_rows = []  # log of failed files

# --- 2. Loop all of the provinces ---
for prov in prov_list:
    kode_prov = prov["kode_prov"]
    nama_prov = prov["nama_prov"]

    print(f"\n=== Provinsi {nama_prov} (kode {kode_prov}) ===")

    # take this page for the dropdown reading
    params_page = {"tahun": tahun, "provinsi": kode_prov, "pemda": "--"}
    resp_page = session.get(base_page_url, params=params_page, timeout=60, verify=False)
    resp_page.raise_for_status()

    soup = BeautifulSoup(resp_page.text, "lxml")
    pemda_select = soup.find("select", {"name": "pemda"}) or soup.find("select", {"id": "pemda"})

    if pemda_select is None:
        print(f"  ⚠️ Tidak ada dropdown pemda, skip provinsi ini.")
        gagal_rows.append({
            "tahun": tahun,
            "kode_prov": kode_prov,
            "nama_prov": nama_prov,
            "kode_pemda": None,
            "nama_pemda": None,
            "status": "no-pemda-dropdown"
        })
        continue

    pemda_list = []
    for opt in pemda_select.find_all("option"):
        val = (opt.get("value") or "").strip()
        text = opt.text.strip()
        if not val or val == "--":
            continue
        pemda_list.append({"kode_pemda": val, "nama_pemda": text})

    print(f"  ➜ {len(pemda_list)} pemda ditemukan.")

    # --- 3. Looping all of the districs at these provinces ---
    for item in pemda_list:
        kode_pemda = item["kode_pemda"]
        nama_pemda = item["nama_pemda"]

        print(f"  Mengambil: {nama_pemda} (kode {kode_pemda}) ...")

        resp_csv = session.get(
            base_csv_url,
            params={"type": "tkdd", "tahun": tahun, "provinsi": kode_prov, "pemda": kode_pemda},
            timeout=60,
            verify=False,
        )

        if resp_csv.status_code != 200:
            print(f"     ⚠️ Gagal (status {resp_csv.status_code}), skip.")
            gagal_rows.append({
                "tahun": tahun,
                "kode_prov": kode_prov,
                "nama_prov": nama_prov,
                "kode_pemda": kode_pemda,
                "nama_pemda": nama_pemda,
                "status": resp_csv.status_code
            })
            continue

        df = pd.read_csv(io.StringIO(resp_csv.text))

        # file name: 2022_Kab. Buton Selatan.xlsx --- adjustable based on your interest
        base_name = f"{tahun}_{safe_filename(nama_pemda)}.xlsx"
        path_file = os.path.join(root_folder, base_name)

        # anticipate the duplication files
        counter = 2
        while os.path.exists(path_file):
            new_name = f"{tahun}_{safe_filename(nama_pemda)}_{counter}.xlsx"
            path_file = os.path.join(root_folder, new_name)
            counter += 1

        df.to_excel(path_file, index=False)
        print(f"     ➜ Disimpan: {os.path.basename(path_file)}")

# --- 4. Save the failed log files (if any)---
if gagal_rows:
    df_gagal = pd.DataFrame(gagal_rows)
    log_name = f"tkdd_{tahun}_log_gagal.xlsx"
    df_gagal.to_excel(log_name, index=False)
    print(f"\n⚠️ Ada pemda yang gagal diambil. Log disimpan sebagai: {log_name}")
else:
    print("\n✅ Semua pemda berhasil diambil tanpa error HTTP.")

# --- 5. Zip all of the folders of kab/kota ---
zip_name = f"{root_folder}.zip"
shutil.make_archive(root_folder, "zip", root_folder)
print(f"\n🎁 ZIP kab/kota seluruh Indonesia tersimpan sebagai: {zip_name}")


Mengambil daftar provinsi untuk tahun 2025 ...
➜ Ditemukan 38 provinsi.


=== Provinsi Provinsi Aceh (kode 01) ===
  ➜ 24 pemda ditemukan.
  Mengambil: Provinsi Aceh (kode 00) ...
     ➜ Disimpan: 2025_Provinsi Aceh.xlsx
  Mengambil: Kab. Aceh Barat (kode 01) ...
     ➜ Disimpan: 2025_Kab. Aceh Barat.xlsx
  Mengambil: Kab. Aceh Besar (kode 02) ...
     ➜ Disimpan: 2025_Kab. Aceh Besar.xlsx
  Mengambil: Kab. Aceh Selatan (kode 03) ...
     ➜ Disimpan: 2025_Kab. Aceh Selatan.xlsx
  Mengambil: Kab. Aceh Singkil (kode 04) ...
     ➜ Disimpan: 2025_Kab. Aceh Singkil.xlsx
  Mengambil: Kab. Aceh Tengah (kode 05) ...
     ➜ Disimpan: 2025_Kab. Aceh Tengah.xlsx
  Mengambil: Kab. Aceh Tenggara (kode 06) ...
     ➜ Disimpan: 2025_Kab. Aceh Tenggara.xlsx
  Mengambil: Kab. Aceh Timur (kode 07) ...
     ➜ Disimpan: 2025_Kab. Aceh Timur.xlsx
  Mengambil: Kab. Aceh Utara (kode 08) ...
     ➜ Disimpan: 2025_Kab. Aceh Utara.xlsx
  Mengambil: Kab. Bireuen (kode 09) ...
     ➜ Disimpan: 2025_Kab. Bireuen.

In [23]:
# Different code to differentiate the name of tkdd accounts, as DJPK has updated several acounts name since 2018.

import requests
from bs4 import BeautifulSoup
import pandas as pd
import io
import os
import shutil
import re
import urllib3

# Turn off warning SSL cuz verify=False
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# ============================================================
# >>> Need to be replaced <<<
tahun = 2017   # selected year only
# ============================================================

base_page_url = "https://djpk.kemenkeu.go.id/portal/data/tkdd"
base_csv_url = "https://djpk.kemenkeu.go.id/portal/csv"

# ------------------------------------------------------------
# Update accounts name of 2017 -> latest accounts name (2018 - up)
# (only need to erase the "Transfer ", but if
# there's the updates one, could be added)
# ------------------------------------------------------------
mapping_akun_2017_ke_2023 = {
    # ====== DBH / DANA BAGI HASIL ======
    # "accounts name in 2017" : "standard accounts name in pivot"

    # more likely like these:
    "DBH Gas Bumi 0,5%": "DBH SDA Gas Bumi 0,5%",
    "DBH Minyak Bumi 0,5%": "DBH SDA Minyak Bumi 0,5%",
    "DBH Minyak Bumi 15%": "DBH SDA Minyak Bumi 15%",
    "DBH Cukai Hasil Tembakau": "DBH Cukai Hasil Tembakau",
    "DBH Pertambangan Umum - Iuran Tetap": "DBH SDA Minerba - Iuran Tetap",
    "DBH Pertambangan Umum - Royalti": "DBH SDA Minerba - Royalti",
    "DBH Panas Bumi - Iuran Produksi": "DBH SDA Panas Bumi - Iuran Produksi",
    "DBH Panas Bumi - Iuran Tetap": "DBH SDA Panas Bumi - Iuran Tetap",
    "DBH Panas Bumi - Setoran Bagian Pemerintah": "DBH SDA Panas Bumi - Setoran Bagian Pemerintah",
    "DBH Kehutanan - PSDH": "DBH SDA Kehutanan - PSDH",
    "DBH Kehutanan - Dana Reboisasi": "DBH SDA Kehutanan - Dana Reboisasi",
    "DBH PPh Pasal 21": "DBH PPh Pasal 21",
    "DBH PPh Pasal 25/29 OP": "DBH PPh Pasal 25/29 OP",
    "DBH Perikanan": "DBH SDA Perikanan",
    "DBH Gas Bumi Otonomi Khusus": "DBH SDA Gas Bumi Otonomi Khusus",
    "DBH Minyak Bumi Otonomi Khusus": "DBH SDA Minyak Bumi Otonomi Khusus",

    # ====== DANA ALOKASI UMUM ======
    "Dana Alokasi Umum": "Dana Alokasi Umum",
    "Dana Alokasi Umum Bidang Kesehatan": "Dana Alokasi Umum Bidang Kesehatan",
    "Dana Alokasi Umum Bidang Pendidikan": "Dana Alokasi Umum Bidang Pendidikan",
    "Dana Alokasi Umum Bidang Pekerjaan Umum": "Dana Alokasi Umum Bidang Pekerjaan Umum",
    "Dana Alokasi Umum Pendanaan Kelurahan": "Dana Alokasi Umum Pendanaan Kelurahan",
    "Dana Alokasi Umum Penggajian Formasi PPPK": "Dana Alokasi Umum Penggajian Formasi PPPK",

    # ====== DAK FISIK ======
    "Dana Alokasi Khusus Fisik": "Dana Alokasi Khusus Fisik",
    "Dana Alokasi Khusus Penugasan": "Dana Alokasi Khusus Penugasan",

    # ====== DANA INSENTIF DAERAH ======
    "Dana Insentif Daerah": "Dana Insentif Daerah",

    # ====== DANA OTONOMI KHUSUS (PAPUA, ACEH, DLL.) ======
    "Belanja Dana Otonomi Khusus yang Telah Ditentukan Penggunaannya Provinsi Papua":
        "Belanja Dana Otonomi Khusus yang Telah Ditentukan Penggunaannya Provinsi Papua",
    "Dana Otonomi Khusus Provinsi Aceh": "Dana Otonomi Khusus Provinsi Aceh",
    # etc.
    # others:
    "Belanja Dana Otonomi Khusus yang Bersifat Umum Provinsi Papua":
        "Belanja Dana Otonomi Khusus yang Bersifat Umum Provinsi Papua",
    "Belanja Dana Tambahan Infrastruktur Provinsi Papua":
        "Belanja Dana Tambahan Infrastruktur Provinsi Papua",
    # etc.

    # ====== DANA KEISTIMEWAAN DIY ======
    "Dana Keistimewaan DIY": "Dana Keistimewaan Daerah Istimewa Yogyakarta",
    "Dana Keistimewaan Daerah Istimewa Yogyakarta":
        "Dana Keistimewaan Daerah Istimewa Yogyakarta",

    # ====== DAK NONFISIK ======
    "Dana Bantuan Operasional Sekolah": "Dana Bantuan Operasional Sekolah",
    "Dana Tunjangan Profesi Guru ASN Daerah": "Dana Tunjangan Profesi Guru ASN Daerah",
    "Dana Tambahan Penghasilan Guru ASN Daerah": "Dana Tambahan Penghasilan Guru ASN Daerah",
    "Dana Bantuan Operasional Keluarga Berencana": "Dana Bantuan Operasional Keluarga Berencana",
    "Dana Pelayanan Administrasi Kependudukan": "Dana Pelayanan Administrasi Kependudukan",
    "Dana Pelayanan Kepariwisataan": "Dana Pelayanan Kepariwisataan",
    "Dana Ketahanan Pangan dan Pertanian": "Dana Ketahanan Pangan dan Pertanian",
    "Dana Fasilitasi Penanaman Modal": "Dana Fasilitasi Penanaman Modal",
    "Dana Peningkatan Kapasitas Ketenagakerjaan": "Dana Peningkatan Kapasitas Ketenagakerjaan",
    "Dana Pelayanan Perlindungan Perempuan dan Anak": "Dana Pelayanan Perlindungan Perempuan dan Anak",
    "Dana Bantuan Operasional Kesehatan": "Dana Bantuan Operasional Kesehatan",
    "Dana Peningkatan Kapasitas Koperasi dan Usaha Mikro Kecil":
        "Dana Peningkatan Kapasitas Koperasi dan Usaha Mikro Kecil",
    "Dana Peningkatan Kapasitas Koperasi": "Dana Peningkatan Kapasitas Koperasi",
    "Dana Penguatan Kapasitas Kelembagaan Sentra Industri Kecil dan Menengah":
        "Dana Penguatan Kapasitas Kelembagaan Sentra Industri Kecil dan Menengah",
    "Dana Bantuan Biaya Layanan Pengolahan Sampah (BLPS)":
        "Dana Bantuan Biaya Layanan Pengolahan Sampah (BLPS)",
    "Dana Bantuan Pengembangan Program (BPP) Perpustakaan Daerah":
        "Dana Bantuan Pengembangan Program (BPP) Perpustakaan Daerah",
    "Dana Bantuan Operasional Penyelenggaraan (BOP) Pendidikan Kesetaraan":
        "Dana Bantuan Operasional Penyelenggaraan (BOP) Pendidikan Kesetaraan",
    "Dana Bantuan Operasionaal Penyelenggaraan (BOP) Museum dan Taman Budaya":
        "Dana Bantuan Operasional Penyelenggaraan (BOP) Museum dan Taman Budaya",
    "Dana Bantuan Operasional Penyelenggaraan - Pendidikan Anak Usia Dini":
        "Dana Bantuan Operasional Penyelenggaraan - Pendidikan Anak Usia Dini",

    # ====== TRANSFER HIBAH ======
    "Transfer Hibah Pinjaman Luar Negeri": "Transfer Hibah Pinjaman Luar Negeri",
    "Transfer Hibah Dalam Negeri": "Transfer Hibah Dalam Negeri",
    "Transfer Hibah Hibah Luar Negeri": "Transfer Hibah Hibah Luar Negeri",

    # ====== DANA DESA & TOTAL ======
    "Dana Desa": "Dana Desa",
    "TOTAL TKDD": "TOTAL TKDD",

    # Adjustable. I only concern to DBH SDA.
}

# ------------------------------------------------------------
# Session HTTP to stably connection
# ------------------------------------------------------------
session = requests.Session()
session.headers.update({
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0 Safari/537.36"
    )
})

def safe_filename(name: str) -> str:
    """
    Bikin nama file aman untuk Windows:
    - tetap pakai spasi
    - tetap pakai 'Kab.' / 'Kota'
    - hapus karakter terlarang: \ / : * ? " < > |
    """
    name = re.sub(r'[\\/*?:"<>|]', "", name)
    return name.strip()

print(f"Mengambil daftar provinsi untuk tahun {tahun} ...")

# ------------------------------------------------------------
# 1. Take the provinces lists from the root page
# ------------------------------------------------------------
params_root = {"tahun": tahun, "provinsi": "--", "pemda": "--"}
resp_root = session.get(base_page_url, params=params_root, timeout=60, verify=False)
resp_root.raise_for_status()

soup_root = BeautifulSoup(resp_root.text, "lxml")
prov_select = soup_root.find("select", {"name": "provinsi"}) or soup_root.find("select", {"id": "provinsi"})

prov_list = []
for opt in prov_select.find_all("option"):
    val = (opt.get("value") or "").strip()
    text = opt.text.strip()
    if not val or val == "--":
        continue
    prov_list.append({"kode_prov": val, "nama_prov": text})

print(f"➜ Ditemukan {len(prov_list)} provinsi.\n")

# Root folder to all of the kab/kota in this year
root_folder = f"tkdd_{tahun}_kab_kota"
os.makedirs(root_folder, exist_ok=True)

gagal_rows = []  # Log the failed files

# ------------------------------------------------------------
# 2. Loop all of the provinces
# ------------------------------------------------------------
for prov in prov_list:
    kode_prov = prov["kode_prov"]
    nama_prov = prov["nama_prov"]

    print(f"\n=== Provinsi {nama_prov} (kode {kode_prov}) ===")

    # Take the province page to read the districts dropdown
    params_page = {"tahun": tahun, "provinsi": kode_prov, "pemda": "--"}
    resp_page = session.get(base_page_url, params=params_page, timeout=60, verify=False)
    resp_page.raise_for_status()

    soup = BeautifulSoup(resp_page.text, "lxml")
    pemda_select = soup.find("select", {"name": "pemda"}) or soup.find("select", {"id": "pemda"})

    if pemda_select is None:
        print("  ⚠️ Tidak ada dropdown pemda, skip provinsi ini.")
        gagal_rows.append({
            "tahun": tahun,
            "kode_prov": kode_prov,
            "nama_prov": nama_prov,
            "kode_pemda": None,
            "nama_pemda": None,
            "status": "no-pemda-dropdown"
        })
        continue

    pemda_list = []
    for opt in pemda_select.find_all("option"):
        val = (opt.get("value") or "").strip()
        text = opt.text.strip()
        if not val or val == "--":
            continue
        pemda_list.append({"kode_pemda": val, "nama_pemda": text})

    print(f"  ➜ {len(pemda_list)} pemda ditemukan.")

    # --------------------------------------------------------
    # 3. Loop all of the district files in this province
    # --------------------------------------------------------
    for item in pemda_list:
        kode_pemda = item["kode_pemda"]
        nama_pemda = item["nama_pemda"]

        print(f"  Mengambil: {nama_pemda} (kode {kode_pemda}) ...")

        resp_csv = session.get(
            base_csv_url,
            params={"type": "tkdd", "tahun": tahun, "provinsi": kode_prov, "pemda": kode_pemda},
            timeout=60,
            verify=False,
        )

        if resp_csv.status_code != 200:
            print(f"     ⚠️ Gagal (status {resp_csv.status_code}), skip.")
            gagal_rows.append({
                "tahun": tahun,
                "kode_prov": kode_prov,
                "nama_prov": nama_prov,
                "kode_pemda": kode_pemda,
                "nama_pemda": nama_pemda,
                "status": resp_csv.status_code
            })
            continue

        # --- Read CSV ---
        df = pd.read_csv(io.StringIO(resp_csv.text))

        # ----------------------------------------------------
        # 3a. Normalised the special accounts name in 2017
        # ----------------------------------------------------
        if tahun == 2017 and "akun" in df.columns:
            # ensure string and erase the spaces
            df["akun"] = df["akun"].astype(str).str.strip()

            # 1) delete prefix "Transfer " before the account name
            #    e.g.: "Transfer DBH Gas Bumi" -> "DBH Gas Bumi"
            df["akun"] = df["akun"].str.replace(r"^\s*Transfer\s+", "", regex=True)

            # 2) Apply mapping lists name if any difference
            if mapping_akun_2017_ke_2023:
                df["akun"] = df["akun"].replace(mapping_akun_2017_ke_2023)

        # ----------------------------------------------------
        # 3b. Save excel files per kab/kota
        # ----------------------------------------------------
        base_name = f"{tahun}_{safe_filename(nama_pemda)}.xlsx"
        path_file = os.path.join(root_folder, base_name)

        # if any duplicate files (adding _2 , etc)
        counter = 2
        while os.path.exists(path_file):
            new_name = f"{tahun}_{safe_filename(nama_pemda)}_{counter}.xlsx"
            path_file = os.path.join(root_folder, new_name)
            counter += 1

        df.to_excel(path_file, index=False)
        print(f"     ➜ Disimpan: {os.path.basename(path_file)}")

# ------------------------------------------------------------
# 4. Save log failed files (if any)
# ------------------------------------------------------------
if gagal_rows:
    df_gagal = pd.DataFrame(gagal_rows)
    log_name = f"tkdd_{tahun}_log_gagal.xlsx"
    df_gagal.to_excel(log_name, index=False)
    print(f"\n⚠️ Ada pemda yang gagal diambil. Log disimpan sebagai: {log_name}")
else:
    print("\n✅ Semua pemda berhasil diambil tanpa error HTTP.")

# ------------------------------------------------------------
# 5. Zip all of the folders
# ------------------------------------------------------------
zip_name = f"{root_folder}.zip"
shutil.make_archive(root_folder, "zip", root_folder)
print(f"\n🎁 ZIP kab/kota seluruh Indonesia tersimpan sebagai: {zip_name}")


Mengambil daftar provinsi untuk tahun 2017 ...
➜ Ditemukan 34 provinsi.


=== Provinsi Prov. Aceh (kode 01) ===
  ➜ 24 pemda ditemukan.
  Mengambil: Prov. Aceh (kode 00) ...
     ➜ Disimpan: 2017_Prov. Aceh_2.xlsx
  Mengambil: Kab. Aceh Barat (kode 01) ...
     ➜ Disimpan: 2017_Kab. Aceh Barat_2.xlsx
  Mengambil: Kab. Aceh Besar (kode 02) ...
     ➜ Disimpan: 2017_Kab. Aceh Besar_2.xlsx
  Mengambil: Kab. Aceh Selatan (kode 03) ...
     ➜ Disimpan: 2017_Kab. Aceh Selatan_2.xlsx
  Mengambil: Kab. Aceh Singkil (kode 04) ...
     ➜ Disimpan: 2017_Kab. Aceh Singkil_2.xlsx
  Mengambil: Kab. Aceh Tengah (kode 05) ...
     ➜ Disimpan: 2017_Kab. Aceh Tengah_2.xlsx
  Mengambil: Kab. Aceh Tenggara (kode 06) ...
     ➜ Disimpan: 2017_Kab. Aceh Tenggara_2.xlsx
  Mengambil: Kab. Aceh Timur (kode 07) ...
     ➜ Disimpan: 2017_Kab. Aceh Timur_2.xlsx
  Mengambil: Kab. Aceh Utara (kode 08) ...
     ➜ Disimpan: 2017_Kab. Aceh Utara_2.xlsx
  Mengambil: Kab. Bireuen (kode 09) ...
     ➜ Disimpan: 2017_Kab.